In [ ]:
from wpt_tools import wpt_eval
import importlib
importlib.reload(wpt_tools)

### Create instance of analysis and set initial variables

In [ ]:
wpt_eval_ins = wpt_eval.wpt_eval()
rx_port = 1 # 1 or 2

### Import 2-port network data
options
- 'file' option: import touchstone file
- 'measure_pico' option: measure using picovna
    - parameters need to match cal file

In [ ]:
mode = 'file'
#mode = 'measure_pico'

if mode == 'file':
    #wpt_eval_ins.import_touchstone()
    wpt_eval_ins.import_touchstone('./assets/100mmTX_20mmRX_40mmD-1.s2p')

if mode == 'measure_pico':
    # vna registry setting (it's either of these two)
    progid = "PicoControl2.PicoVNA_2"
    # progid = "PicoControl3.PicoVNA_3"

    # Export setting
    touchstone_export = 'export_touchstone/test'

    # Need to match calibration setup
    cal_file = '../sample_cal/1_to_10MHz.cal'
    start_f = 1e6 #[Hz]
    end_f = 10e6 #[Hz]
    sweep_points = 1001
    power_level = -10 #[dBm]
    RBW = 1e3 #[Hz]
    z0 = 50

    wpt_eval_ins.picoVNA_measure(cal_file=cal_file, start_f=start_f, end_f=end_f,\
        sweep_points=sweep_points, power_level=power_level, RBW=RBW, z0=z0, progid=progid)

    wpt_eval_ins.export_touchstone(touchstone_export)

### Plot Z-parameters (full-range)

In [ ]:
wpt_eval_ins.plot_z_full()

### Efficiency and optimal load analysis (for general 2-port networks)
Reference: Y. Narusue, et al., "Load optimization factors for analyzing the efficiency of wireless power transfer systems using two-port network parameters," IEICE ELEX, 2020.

Unstable when far from resonant frequency (probably because to S to Z conversion becomes unstable)

In [ ]:
# For maximum efficiency analysis
target_f = 6.78e6
range_f = 1e6

wpt_eval_ins.set_f_target_range(target_f=target_f, range_f=range_f)
f_opt, eff_opt, r_opt, x_opt = wpt_eval_ins.efficiency_load_analysis(show_plot=1, rx_port=rx_port)

### Curve-fitting and Z-matrix plot (only for 2-port measurement of Series-Series LCR coupling)

In [ ]:
# For maximum efficiency analysis
target_f = 6.78e6
range_f = 1e6

wpt_eval_ins.set_f_target_range(target_f=target_f, range_f=range_f)
wpt_eval_ins.plot_z_narrow_fit()

### Optimal load visualization
Imura, "Wireless Power Transfer: Using Magnetic and Electric Resonance Coupling Techniques," Springer Singapore 2020.

In [ ]:
wpt_eval_ins.optimal_load_plot(min_rez=0.1, max_rez=50, step_rez=0.2,\
    min_imz=-200, max_imz=200, step_imz=1, input_voltage=5, rx_port=rx_port)

### Capacitor filter calculator
![](./img/CpCsRltopology.png)

In [ ]:
wpt_eval_ins.set_f_target_range(target_f=target_f, range_f=range_f)
wpt_eval_ins.rxc_filter_calc(rx_port = rx_port, rload = 100, c_network = 'CpCsRl')